In [1]:
import getpass
import os

os.environ["MISTRAL_API_KEY"] = getpass.getpass()

# Installation

In [2]:
!pip install --quiet --upgrade langchain langchain-community langchain-chroma
!pip install --quiet langchain-openai
!pip install --quiet pypdf
!pip install --quiet sentence-transformers

# Necessary Packages

In [3]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser  # For the chains
from langchain_core.runnables import RunnablePassthrough   # For the chains
#from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader #For PDF Loader
from langchain_mistralai import ChatMistralAI #For MISTRAL MODEL

USER_AGENT environment variable not set, consider setting it to identify your requests.


# Selected Model: Mistral 7B

We have selected the **Mistral model** with **7 billion parameters**, and we are accessing it remotely using a Mistral API Key.

## Reasons for Selection
1. **Open-Source**: Being open-source allows for customization and adaptability.
2. **Relatively Small Size**: With 7B parameters, it offers a good balance between performance and computational cost, making it feasible to use locally with proper quantization.
3. **Proven Performance**: The model has shown excellent results across various tasks, proving effective in multiple contexts.

*Mistral context window has 32.8k size which is proximatly 20,000–25,000 words*

In [4]:
llm = ChatMistralAI(model="open-mistral-7b")

# Indexing: Load (PDF)
We use PyPDFLoader for loading local pdf, but we mihgt change that for web Documents loadin, later with  **WebBaseLoader**

In [33]:
from langchain.document_loaders import PyPDFLoader
# Replace this with the path to your local PDF file
pdf_file_path = "ArtificialIntelligenceAct-1-50.pdf"
# Load the local PDF file
loader = PyPDFLoader(pdf_file_path)
# Load and process the document
docs = loader.load()

len(docs[0].page_content) #here docs is already an LangChain Object

1882

# Indexing: Split

we use Chroma as our Vector Store
we use all-MiniLM-L6-v2 from Microsoftmodel to create the Embeddings. (OpenAI are pay to use)

The chunk size is an balanced Value considering on the Mistral and embedding model context window size which seems to work good on practice


# Indexing: Split

we use Chroma as our Vector Store
we use all-MiniLM-L6-v2 from Microsoftmodel to create the Embeddings. (OpenAI are pay to use)

The chunk size is an balanced Value considering on the Mistral and embedding model context window size which seems to work good on practice


In [34]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

print(f"number of chunks: {len(splits)}")

number of chunks: 115


# Indexing: Store Embeddings
- **Chroma** is an open-source **vector database** that’s designed for scalable, high-performance **similarity search**.
- The model **all-MiniLM-L6-v2** is part of the MiniLM (Mini Language Models) family developed by **Microsoft**. Especially suited for **semantic similarity** tasks, **sentence embedding**, and **question-answer retrieval**.

In [7]:
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))

C:\Users\perei\AppData\Local\Temp\ipykernel_3556\1528764191.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))
c:\Users\perei\miniconda3\envs\chatbot\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


#  Retrieval and Generation: Retrieve
## Retreiver:
We're using the most common type of Retriever wich is the VectorStoreRetriever.
## Prompt:
The prompt that is being pulled from *https://smith.langchain.com/hub/rlm/rag-prompt*: 

"
`HUMAN`

`You are an assistant for question-answering tasks. Use the following retrieved context to answer the question. If you don't know the answer, state that clearly. Limit your response to three sentences, keeping the answer concise.`


`Question: {question}`

`Context: {context}`

`Answer:`
"

In [30]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(
                search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.0}
            )
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

c:\Users\perei\miniconda3\envs\chatbot\Lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


# Retrieval and Generation: Generate

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()  #because some meta_data might come along with the text. Only extract the text
)

query = "How does the AI Act aim to regulate high-risk AI systems?"
relevant_docs = retriever.get_relevant_documents(query)

print("Number of relevant documents:", len(relevant_docs), end="\n\n") # Default is 4

for i, doc in enumerate(relevant_docs, start=1):
    print(f"Document {i}:")
    print(doc.page_content)
    print("-" * 80)

rag_chain.invoke(query)


[Document(metadata={'page': 47, 'source': 'ArtificialIntelligenceAct-1-50.pdf'}, page_content='(52) As regards stand-alone AI systems, namely high-risk AI systems other than those that are \nsafety components of products, or that are themselves products, it is appropriate to classify \nthem as high-risk if, in light of their intended purpose, they pose a high risk of harm to the \nhealth and safety or the fundamental rights of persons, taking into account both the severity \nof the possible harm and its probability of occurrence and they are used in a number of \nspecifically pre-defined areas specified in this Regulation. The identification of those \nsystems is based on the same methodology and criteria envisaged also for any future \namendments of the list of high-risk AI systems that the Commission should be \nempowered to adopt, via delegated acts, to take into account the rapid pace of \ntechnological development, as well as the potential changes in the use of AI systems.'), Docu

"The AI Act aims to regulate high-risk AI systems by classifying them as such if they pose a high risk of harm to health, safety, or fundamental rights, considering both the severity and probability of harm. Providers of non-high-risk AI systems should document their assessment and register the system in an EU database. For clarity, the Commission may provide guidelines and examples. The classification of an AI system as high-risk does not necessarily mean the same for the product it's a part of, depending on the relevant Union harmonization legislation."

## Description of the Chatbot Interface

The chatbot interface was developed using the **Streamlit** library, a popular tool for rapidly and interactively creating web applications. Below are the main components and functionalities of the interface:

1. **Loading Environment Variables**:
   - The code uses the `dotenv` library to load environment variables, such as the API key for the language model (Mistral), allowing the application to securely access sensitive configurations.

2. **Model Initialization**:
   - The `ChatMistralAI` language model is initialized with the API key. This model is responsible for generating responses based on user questions.

3. **Loading and Processing Documents**:
   - Users can upload documents in PDF format. These files are processed into smaller parts (chunks) to facilitate the retrieval of relevant information during interactions.

4. **Message History**:
   - The application maintains a history of messages to provide context for responses. The history is limited to a maximum number of responses (set to 5), ensuring that previous interactions are considered without overloading the model's memory.

5. **User Input**:
   - The interface presents an input field where users can type their questions. Once a question is submitted, the interface records the message and calls the function responsible for generating a response.

6. **Message Display**:
   - The messages exchanged between the user and the chatbot are displayed in the interface in a chat format, using the `st.chat_message` function to enhance readability.

7. **Dynamic Interaction**:
   - The interface responds dynamically to file uploads and message submissions. If a file is successfully uploaded, the chatbot can generate responses based on the content of that file in response to user questions.

8. **Error Handling**:
   - If no file has been uploaded and the user attempts to submit a question, a message will be displayed prompting the user to upload a file before continuing.

This approach provides an interactive and user-friendly experience, allowing users to query data contained in the documents they upload and receive informative responses from the chatbot seamlessly.
